<a href="https://colab.research.google.com/github/ugonfor/SimpleCNN/blob/main/SimpleCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import torch.nn.functional as F
import torch.optim as optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# MNIST Dataset
# Training 데이터 저장
train_dataset = datasets.MNIST(root='./MNIST_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

# Test 데이터 저장
test_dataset = datasets.MNIST(root='./MNIST_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data 로드
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN,self).__init__()
        # (input_channel_size, output_channel_size, kernel_size, stride, padding, ...) 
        # input_channel_size : Input Image의 depth (ex, RGB -> depth : 3. So, input_chennel=3)
        # output_channel_size : output으로 만들 필터의 개수 (ex, 필터를 10개를 만든다 -> output_chennel=10)
        # kernel_size : 필터의 사이즈 (ex, kernel_size=5 -> 5*5 size filter)
        # padding : 패딩
        # stride : 한번에 가는 개수
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5, padding=1)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) #MNIST의 경우 가장자리에 데이터가 거의 없어 padding 필요 X

        #MaxPooling -> n*n에서 가장 큰 값을 택하는 방법
        self.mp = nn.MaxPool2d(2)

        #Fully Connected Layer
        #여기 오류한번 내면 자동으로 계산해줌
        self.fc = nn.Linear(320, 10)
      
    def forward(self, x):
        #input값의 차원 -> 첫번째 차원
        input_size = x.size(0)

        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(input_size, -1) #텐서를 모두 펼치는 작업
        x = self.fc(x) 
        return F.log_softmax(x,-1) 

model = SimpleCNN()

criterion = nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
# optimizer
# lr : learning rate
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5) 

def train(epoch):
    #model을 train mode로
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # 변수 선언
        data, target = data.to(device), target.to(device)
        #초기화
        optimizer.zero_grad()

        #모델
        output = model(data)
        
        #loss 계산, negative log likelihood loss
        # cross_entropy를 사용하면 log_softmax부터 아래까지 한번에 가능하긴 함
        loss = criterion(output, target) 
        loss.backward() # 역전파
        #모델에 대해서 변화를 주는 과정
        optimizer.step()
        
        if batch_idx % 10 == 0: 
            #batch 10개마다 정보 출력
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test():
    #model을 evaluation mode로
    model.eval()

    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
if __name__ == '__main__':
    for epoch in range(1, 10):
        train(epoch)
        test()

